In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
labels_csv = {'train': "/scratch/scratch6/akansh12/DeepEXrays/physionet.org/files/vindr-cxr/1.0.0/annotations/image_labels_train.csv",
             'test': "/scratch/scratch6/akansh12/DeepEXrays/physionet.org/files/vindr-cxr/1.0.0/annotations/image_labels_test.csv"
             }

In [3]:
train_csv_orig = pd.read_csv(labels_csv['train'])
train_csv_orig.head(10)

,image_id,rad_id,Aortic enlargement,Atelectasis,Calcification,Cardiomegaly,Clavicle fracture,Consolidation,Edema,Emphysema,...,Pneumothorax,Pulmonary fibrosis,Rib fracture,Other lesion,COPD,Lung tumor,Pneumonia,Tuberculosis,Other diseases,No finding
0,000434271f63a053c4128a0ba6352c7f,R2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,000434271f63a053c4128a0ba6352c7f,R3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,000434271f63a053c4128a0ba6352c7f,R6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,00053190460d56c53cc3e57321387478,R11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,00053190460d56c53cc3e57321387478,R2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,00053190460d56c53cc3e57321387478,R7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,0005e8e3701dfb1dd93d53e2ff537b6e,R10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
7,0005e8e3701dfb1dd93d53e2ff537b6e,R8,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
8,0005e8e3701dfb1dd93d53e2ff537b6e,R9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0
9,0006e0a85696f6bb578e84fafa9a5607,R11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
test_csv_orig = pd.read_csv(labels_csv['test'])
test_csv_orig.tail()

,image_id,Aortic enlargement,Atelectasis,Calcification,Cardiomegaly,Clavicle fracture,Consolidation,Edema,Emphysema,Enlarged PA,...,Pneumothorax,Pulmonary fibrosis,Rib fracture,Other lesion,COPD,Lung tumor,Pneumonia,Tuberculosis,Other diseases,No finding
2995,a039af299f86007d0d77da077a6def9a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2996,aba3d1f5b1c04236f52a8980929b2cfa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2997,6d3d6b53f358a983b486e9e03144eb62,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2998,d6678cb7ae39f575d35ab9da6d7cb171,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2999,e82d33dfd04d95c61d8d133dcbf29691,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
test_ann_abnornal_vs_normal =  pd.DataFrame(columns=['image_id', 'Abnormality'])
test_ann_abnornal_vs_normal['image_id'] = test_csv_orig['image_id']
test_ann_abnornal_vs_normal['Abnormality'] = 1- test_csv_orig['No finding']


In [6]:
test_ann_abnornal_vs_normal.to_csv("./Radiologist_selction_csv_files/test_abnormal_vs_normal.csv", index=None)

In [7]:
import numpy as np
import torch
from PIL import Image
import pandas as pd
import os
import matplotlib.pyplot as plt
torch.manual_seed(0)
from glob import glob

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision import transforms, models, datasets

from PIL import Image

from torch import nn
from torch import optim
from tqdm.notebook import tqdm

In [8]:
from torchsummary import summary

In [59]:
data_transforms = { 
    "train": transforms.Compose([
#         transforms.Grayscale(),
        transforms.Resize((320,320)),
        transforms.RandomHorizontalFlip(p = 0.5), 
        transforms.RandomPerspective(distortion_scale=0.3),
        transforms.RandomRotation((-30,30)),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485], std = [0.229])
    ]),
    
    "test": transforms.Compose([
#         transforms.Grayscale(),
        transforms.Resize((320,320)),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485], std = [0.229])        
    ])
    
}

### Experiment 1

Week annotations

In [60]:
exp_1_week_ann_train = pd.DataFrame(columns=['image_id', 'Abnormality'])
exp_1_week_ann_train['image_id'] = train_csv_orig['image_id']
exp_1_week_ann_train['Abnormality'] = 1- train_csv_orig['No finding']
exp_1_week_ann_train.head(10)

,image_id,Abnormality
0,000434271f63a053c4128a0ba6352c7f,0
1,000434271f63a053c4128a0ba6352c7f,0
2,000434271f63a053c4128a0ba6352c7f,0
3,00053190460d56c53cc3e57321387478,0
4,00053190460d56c53cc3e57321387478,0
5,00053190460d56c53cc3e57321387478,0
6,0005e8e3701dfb1dd93d53e2ff537b6e,1
7,0005e8e3701dfb1dd93d53e2ff537b6e,1
8,0005e8e3701dfb1dd93d53e2ff537b6e,1
9,0006e0a85696f6bb578e84fafa9a5607,0


In [61]:
exp_1_week_ann_train.to_csv("./Radiologist_selction_csv_files/exp_1_week_ann_train.csv", index=None)

In [62]:
class normal_vs_abnormal_dataset(Dataset):
    def __init__(self, label_loc, img_location, transform,  data_type= 'train'):
        label_dataframe = pd.read_csv(label_loc)
        label_dataframe.set_index("image_id", inplace = True)
        filenames = label_dataframe.index.values
        self.full_filename = [os.path.join(img_location,i+'.png') for i in filenames]
        self.labels = label_dataframe.iloc[:,0].values
        self.transform = transform
    def __len__(self):
        return len(self.full_filename)
    
    def __getitem__(self, idx):
        
        image = Image.open(self.full_filename[idx])
        image = self.transform(image)
        return image, np.expand_dims(np.array(self.labels[idx]), axis = 0)

In [63]:
train_data = normal_vs_abnormal_dataset("/storage/home/akansh12/Vin-ChestXR-Abnormality-detection/Notebooks/normal_vs_abnormal/Radiologist_selction_csv_files/exp_1_week_ann_train.csv",
                                       img_location = "/scratch/scratch6/akansh12/DeepEXrays/data/data_1024//train/", transform =data_transforms['train'])
test_data = normal_vs_abnormal_dataset("/storage/home/akansh12/Vin-ChestXR-Abnormality-detection/Notebooks/normal_vs_abnormal/Radiologist_selction_csv_files/test_abnormal_vs_normal.csv",
                                       img_location = "/scratch/scratch6/akansh12/DeepEXrays/data/data_256/test/", transform =data_transforms['test'])

In [64]:
trainloader = DataLoader(train_data,batch_size = 1,shuffle = True)
testloader = DataLoader(test_data,batch_size = 8,shuffle = False)

In [65]:
for x, y in trainloader:
    print(x.shape)
    print(y)
    break

torch.Size([1, 3, 320, 320])
tensor([[1]])


In [66]:
def train_one_epoch(model, optimizer, lr_scheduler,
                    dataloader, epoch, criterion, device):
    
    print("Start Train ...")
    model.train()

    losses_train = []
    model_train_result = []
    train_target = []


    for data, targets in tqdm(dataloader):
        data = data.to(device)
        targets = targets.to(device).type(torch.float)


        outputs = model(data)
        model_train_result.extend(outputs.detach().cpu().numpy().tolist())
        train_target.extend(targets.cpu().numpy())


        loss = criterion(outputs, targets)

        losses_train.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    F1_train = f1_score(train_target, (np.array(model_train_result) > 0.5), average=None)

        
    if lr_scheduler is not None:
        lr_scheduler.step()

    lr = lr_scheduler.get_last_lr()[0]
    print("Epoch [%d]" % (epoch),
          "Mean loss on train:", np.array(losses_train).mean(), 
          "F1 score:",np.array(F1_train),
          "Learning Rate:", lr)

    
    return np.array(losses_train).mean(), np.array(F1_train), lr


def val_epoch(model, dataloader, epoch, criterion, device):
    
    print("Start Validation ...")
    model.eval()
    
    model_val_result = []
    val_target = []
    losses_val = []

    with torch.no_grad():
        for data, targets in tqdm(dataloader):

            data = data.to(device)
            targets = targets.to(device).type(torch.float)

            outputs = model(data)
            
            #loss
            loss = criterion(outputs, targets)
            losses_val.append(loss.item())

            
            model_val_result.extend(outputs.detach().cpu().numpy().tolist())
            val_target.extend(targets.cpu().numpy())
            
        F1_val = f1_score(val_target, (np.array(model_val_result) > 0.5), average=None)




        print("Epoch:  " + str(epoch) + " F1 valid Score:", np.array(F1_val), 
              "Mean Loss", np.array(losses_val).mean())
        
    return np.array(losses_val).mean(), np.array(F1_val)



In [67]:
model = models.densenet121()
from collections import OrderedDict
from torch import nn
model.classifier = nn.Sequential(OrderedDict([
    ('fcl1', nn.Linear(1024,14)),
    ('out', nn.Sigmoid()),
]))
state_dict = torch.load("/scratch/scratch6/akansh12/DeepEXrays/temp/iter_590400.pth.tar", map_location='cpu')['model_state']

for keyA, keyB in zip(state_dict, model.state_dict()):
        state_dict = OrderedDict((keyB if k == keyA else k, v) for k, v in state_dict.items())
model.load_state_dict(state_dict)

model.classifier = nn.Sequential(OrderedDict([
    ('fcl1', nn.Linear(1024,1)),
    ('out', nn.Sigmoid()),
]))

In [73]:
for param in model.parameters():
    param.requires_grad = True
num_epochs = 50
optimizer = optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr = 0.1, steps_per_epoch=1, epochs=50)
# criterion = nn.BCELoss(reduction= 'mean')
criterion = nn.BCELoss()

In [74]:
num_epochs = 50
optimizer = optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr = 0.1, steps_per_epoch=1, epochs=50)

In [76]:
train_loss_history = []
val_loss_history = []
train_f1_history = []
val_f1_history = []
lr_history = []
weights_dir = "/scratch/scratch6/akansh12/DeepEXrays/global_label/efficient-net/"


for epoch in range(num_epochs):
    
    
#     train_loss, train_f1, lr = train_one_epoch(model, optimizer, lr_scheduler,trainloader, epoch, criterion, device = device)
    val_loss, val_f1 = val_epoch(model, testloader, epoch, criterion, device = device)
    
    
    # train history
    train_loss_history.append(train_loss)
    train_f1_history.append(train_f1)
    lr_history.append(lr)
    
    #val history
    val_loss_history.append(val_loss)
    val_f1_history.append(val_f1)
    
    # save best weights
    best_loss = min(val_loss_history)
    if (val_loss <= best_loss) or (epoch % 10 == 0):
        print('saving model')
        torch.save({'state_dict': model.state_dict()},
                    os.path.join(weights_dir, f"A_N_{val_loss:0.6f}_{epoch}_.pth"))


Start Validation ...


  0%|          | 0/375 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Experiment 2

Veto

In [122]:
train_veto_orig = pd.read_csv("/storage/home/akansh12/Vin-ChestXR-Abnormality-detection/Notebooks/weekly_supervised/new_image_labels_train.csv")
exp_2_veto_ann_train = pd.DataFrame(columns=['image_id', 'Abnormality'])

exp_2_veto_ann_train['image_id'] = train_veto_orig['image_id']

exp_2_veto_ann_train['Abnormality'] = 1-train_veto_orig['No finding']

<All keys matched successfully>